In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
os.chdir('../../')

In [ ]:
%cd C:path\to\musicautobot-master #Ändra till din path

In [ ]:
from musicautobot.numpy_encode import *
from musicautobot.utils.file_processing import process_all, process_file
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.multitask_transformer import *
from musicautobot.numpy_encode import stream2npenc_parts

# Generate Music with Pretrained Model

### Load Pretrained

In [ ]:
# Config
config = multitask_config();

# Location of your midi files
midi_path =  Path('data/midi')

# Location of saved datset
data_path = Path('data/numpy')
data_save_name = 'musicitem_data_save.pkl'

In [ ]:
# Data
data = MusicDataBunch.empty(data_path)
vocab = data.vocab

In [ ]:
#pretrained_path = data_path/'models/example.pth'
#pretrained_path = data_path/'models/carcass_model.pth'
pretrained_path = data_path/'models/solos_model.pth'

In [ ]:
# Learner
learn = multitask_model_learner(data, config, pretrained_path=pretrained_path)
# learn.to_fp16();

### Choose existing midi file as a starting point

In [ ]:
example_dir = midi_path/'examples'
midi_files = get_files(example_dir, recurse=True, extensions='.mid'); midi_files[:5]

In [ ]:
file = midi_files[1]; file

In [ ]:
# Encode file 
item = MusicItem.from_file(file, data.vocab)

x = item.to_tensor()
x_pos = item.get_pos_tensor()

In [ ]:
item.show("musicxml")

In [ ]:
item.play()

## Generate

MultitaskTransformer trains on 3 separate tasks. 
1. NextWord
2. Mask
3. Sequence to Sequence

Because we train on 3 separate tasks, we can actually generate some really cool note sequences.

1. NextWord/Autocomplete - Take a sequence of notes and predict the next note
 * 1a. Vanilla Language Model predictions - See [MusicTransformer](../music_transformer) project


2. Mask/Remix - Mask certain parts of song and remix those portions.
 * 2a. Note Masking - Mask all the note pitches and create a new sequence with different notes, but same exact rhythm
 * 2b. Duration Masking - Mask the note durations. Generate a new sequence with the same melody, but with a different rhythm


3. Seq2Seq/Translation - Generate melody from chords or vice versa. 
 * 3a. New Melody - Generate a new melody from existing chords
 * 3b. Harmonization - Generate chords to acompany an existing melody

## 1. NextWord/Autocomplete

Trim the song to only a few notes. Model will use these notes a seed and continue the idea

In [ ]:
seed_len = 6 # 4 beats = 1 bar
seed = item.trim_to_beat(seed_len)

In [ ]:
seed.show()

In [ ]:
pred_nw, full = learn.predict_nw(seed, n_words=200)

In [ ]:
pred_nw.show("musicxml")

In [ ]:
pred_nw.play()

Add more randomness

In [ ]:
pitch_temp = 1.2 # randomness of melody
tempo_temp = 0.8 # randomness or rhythm
top_k = 40
pred_nw_rand, full = learn.predict_nw(seed, temperatures=(pitch_temp, tempo_temp), top_k=top_k, top_p=0.5)
pred_nw_rand.show('musicxml')

In [ ]:
# Convenience function
# out = nw_predict_from_midi(learn, file, seed_len=seed_len, top_k=30, top_p=0.5); out.show()

## 2. Mask/Remix

### 2a. Remix Notes

Mask all the note pitches. Model will create a new song with the same rhythm

In [ ]:
### Mask notes
note_item = item.mask_pitch();

In [ ]:
# Mask vs Original
list(zip(note_item.to_text(None)[:20], item.to_text(None)[:20]))

In [ ]:
pred_note = learn.predict_mask(note_item)

In [ ]:
pred_note.show("musicxml")

In [ ]:
pred_note.play()

### 2b. Remix rhythm

Mask note durations. Same notes, different rhythm

In [ ]:
# duration mask
dur_item = item.mask_duration()

In [ ]:
# Mask vs Original
list(zip(dur_item.to_text(None)[:10], item.to_text(None)[:10]))

In [ ]:
dur_pred = learn.predict_mask(dur_item, temperatures=(0.8,0.8), top_k=40, top_p=0.6)

In [ ]:
dur_pred.show('musicxml')

In [ ]:
# Convenience function
# out = mask_predict_from_midi(learn, file, predict_notes=True)

## 3. Seq2Seq/Translation

Load MultitrackItem.

MultitrackItem keeps track of which notes are part of the melody and which notes are part of the chords.  
This info is needed for translation task

In [ ]:
multitrack_item = MultitrackItem.from_file(file, vocab)

In [ ]:
melody, chords = multitrack_item.melody, multitrack_item.chords

In [ ]:
melody.show('musicxml')

In [ ]:
chords.show('musicxml')

In [ ]:
multitrack_item.play()

## 3a. Create Melody

Use existing chord progression to generate a new melody

In [ ]:
# Use a seed for the melody
partial_melody = melody.trim_to_beat(4)

# Or generate from an empty sequence
empty_melody = MusicItem.empty(vocab, seq_type=SEQType.Melody)

In [ ]:
seed_melody = empty_melody; #seed_melody.show('musicxml')

In [ ]:
pred_melody = learn.predict_s2s(chords, seed_melody, use_memory=True)
pred_melody.show('musicxml')

In [ ]:
pred_melody.play()

In [ ]:
combined = MultitrackItem(pred_melody, chords)
combined.show()

In [ ]:
combined.play()

## 3b. Harmonization

Generate chords to accompany an existing melody

In [ ]:
# partial_chords = chords.trim_to_beat(3);
# partial_chords.show()

empty_chords = MusicItem.empty(vocab, seq_type=SEQType.Chords); empty_chords.show('musicxml')

In [ ]:
pred_chord = learn.predict_s2s(input_item=melody, target_item=empty_chords)

In [ ]:
pred_chord.show('musicxml')

In [ ]:
combined = MultitrackItem(melody, pred_chord)
combined.show('musicxml')

In [ ]:
combined.play()

In [ ]:
# Convenience Function

# out = s2s_predict_from_midi(learn, file, seed_len=10); out.show()